# Mn profiles

In [2]:
import datetime as dt
import glob
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pandas as pd
from pathlib import Path
import xarray as xr

%matplotlib inline

Load data:

In [ ]:
folder  = '/data/brogalla/run_storage/Mn-tuning-202012/'
tuning5 = nc.Dataset(folder+'ref-2015-tuning5/ANHA12-EXH006_2015_monthly.nc')
tuning5.variables['dissolmn']

In [ ]:
drop_vars = [
    "time_centered",
    "time_centered_bounds",
    "time_counter_bounds",
    "nav_lat",
    "bounds_lat",
    "nav_lon",
    "bounds_lon",
    "area",
    "deptht_bounds",
]
chunks = {
    "time_counter": 1,
    "deptht": 1,
}

In [ ]:
def prep_timeseries(dir_path, file_pattern, n_days): 
    ds_files = sorted(dir_path.glob(file_pattern))
    ds = xr.open_mfdataset(
        ds_files[:n_days],
        chunks=chunks,
        drop_variables=drop_vars,
    )
    timeseries = ds.isel(deptht = 1, y = places.PLACES['S3']['NEMO grid ji'][0], 
                             x = places.PLACES['S3']['NEMO grid ji'][1] )
    
    return timeseries

In [ ]:
def get_the_data(dir_path, year, numberdays):
    evenlessink = []
    for filename in sorted(dir_path.glob(f"SalishSea_1d_{year}0[23]*ptrc_T*.nc"))[:numberdays]:
        data = xr.load_dataset(filename)
        data_extract = data.isel(deptht = 1, y = places.PLACES['S3']['NEMO grid ji'][0], 
                             x = places.PLACES['S3']['NEMO grid ji'][1] )
        evenlessink.append(data_extract)
  
    timeseries = xr.concat(evenlessink, dim='time_counter')
    return timeseries

In [ ]:
def get_the_data_numpy(dir_path, year):
    timeseries = {'diatoms': np.zeros(61),
                 'nitrate': np.zeros(61)}
    for it, filename in enumerate(sorted(dir_path.glob(f"SalishSea_1d_{year}0[23]*ptrc_T*.nc"))):
        data = xr.open_dataset(filename)
        timeseries['diatoms'][it] = data['diatoms'].isel(deptht = 1, y = places.PLACES['S3']['NEMO grid ji'][0], 
                             x = places.PLACES['S3']['NEMO grid ji'][1] )[0]
        timeseries['nitrate'][it] = data['nitrate'].isel(deptht = 1, y = places.PLACES['S3']['NEMO grid ji'][0], 
                             x = places.PLACES['S3']['NEMO grid ji'][1] )[0]
        data.close()

    return timeseries

In [ ]:
def get_the_data_nc(dir_path, year):
    timeseries = {'diatoms': np.zeros(61),
                 'nitrate': np.zeros(61)}
    for it, filename in enumerate(sorted(dir_path.glob(f"SalishSea_1d_{year}0[23]*ptrc_T*.nc"))):
        data = xr.open_dataset(filename)
        timeseries['diatoms'][it] = data['diatoms'][0, 1, places.PLACES['S3']['NEMO grid ji'][0], 
                             places.PLACES['S3']['NEMO grid ji'][1]]
        timeseries['nitrate'][it] = data['nitrate'][0, 1, places.PLACES['S3']['NEMO grid ji'][0], 
                             places.PLACES['S3']['NEMO grid ji'][1]]
        data.close()

    return timeseries

In [ ]:
def get_the_data_H201812(year):
    evenlessink = []
    thestring = f'/results/SalishSea/nowcast-green.201812/*/SalishSea_1d_{year}02*ptrc_T*.nc'
    for filename in sorted(glob.glob(thestring)):
        data = xr.open_dataset(filename)
        data_extract = data.isel(deptht = 1, y = places.PLACES['S3']['NEMO grid ji'][0], 
                             x = places.PLACES['S3']['NEMO grid ji'][1] )
        evenlessink.append(data_extract)
        data.close()
    thestring = f'/results/SalishSea/nowcast-green.201812/*/SalishSea_1d_{year}03*ptrc_T*.nc'
    for filename in sorted(glob.glob(thestring)):
        data = xr.open_dataset(filename)
        data_extract = data.isel(deptht = 1, y = places.PLACES['S3']['NEMO grid ji'][0], 
                             x = places.PLACES['S3']['NEMO grid ji'][1] )
        evenlessink.append(data_extract)
        data.close()    

    timeseries = xr.concat(evenlessink, dim='time_counter')
    return timeseries

In [ ]:
directory = '202007C'
dir_path = Path(f"/data/sallen/results/MEOPAR/202007/{directory}")
year = 2015
months_regex = '0[23]'
file_pattern = f"SalishSea_1d_{year}{months_regex}*ptrc_T*.nc"
timeseries2015 = prep_timeseries(dir_path, file_pattern, -1)

In [ ]:
timeseries2015.load(scheduler='processes', num_workers=4)

In [ ]:

fig, ax = plt.subplots(1, 1)
timeseries2015.diatoms.plot(ax=ax, color='g')
ax2 = ax.twinx()
timeseries2015.nitrate.plot(ax=ax2);
ax.grid()